<div class="alert alert-block alert-info">
    
# TOC<a class="anchor"><a id='toc'></a></b><br>
* [<font color='#E8800A'>Naive Bayes</font>](#first-bullet) <br>
- [<font color='#E8800A'>Logistic Regression</font>](#second-bullet)<br>
- [<font color='#E8800A'>KNN</font>](#third-bullet)<br>
- [<font color='#E8800A'>Support Vector Machines</font>](#fourth-bullet)<br>
- [<font color='#E8800A'>Decision Trees</font>](#fifth-bullet)<br>
- [<font color='#E8800A'>Random forest</font>](#sixth-bullet)<br>
- [<font color='#E8800A'>Boosted Trees</font>](#seventh-bullet)<br> 
- [<font color='#E8800A'>Neural Networks</font>](#eighth-bullet)<br> 
- [<font color='#E8800A'>Ensembles</font>](#ninth-bullet)<br>   
    
</div>

In [ ]:
import pandas as pd
import numpy as np
import os
import re



import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

from sklearn.naive_bayes import GaussianNB, ComplementNB, MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC


from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score,\
RandomizedSearchCV
from sklearn.neural_network import MLPClassifier

from sklearn import metrics
from sklearn.metrics import classification_report, f1_score, make_scorer

from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, \
GradientBoostingClassifier, VotingClassifier, AdaBoostClassifier, StackingClassifier

%pip install boruta
from boruta import BorutaPy

%pip install imblearn
from imblearn.over_sampling import SMOTE

%pip install pandas_profiling
from pandas_profiling import ProfileReport

%pip install openpyxl
import openpyxl


In [ ]:
# definitions 
datain_path = 'data/'
src_path = 'src/'


explorations_path = 'explorations/'
submissions_path = 'submissions/'

paths = [explorations_path, submissions_path]
for path in paths:
    if not os.path.exists(path): 
        os.makedirs(path)

In [ ]:
datasets = {
    'train': 'Train.xlsx',
    'test':'Test.xlsx', 
    'both': {
        'train': 'Train.xlsx',   
        'test':'Test.xlsx',
    } 
}

#datasets = pd.DataFrame(datasets, columns=['name', 'path']).set_index('name')

dataset_name = 'both'


In [ ]:
if dataset_name == 'both': 
    data = pd.DataFrame()
    for dataset_path in datasets[dataset_name].values(): 
        tmp = pd.read_excel(os.path.join(datain_path, dataset_path))
        data = pd.concat([data, tmp])

else:    
    dataset_path = datasets[dataset_name]
    data = pd.read_excel(os.path.join(datain_path, dataset_path))
data.head()

In [ ]:
data.isna().sum()

# Explorations

## profile report

profile = ProfileReport(
    data,
    title='Raw data',
    minimal=False, 
    correlations={
    "pearson": {"calculate": True},
    "spearman": {"calculate": False},
    "kendall": {"calculate": False},
    "phi_k": {"calculate": False},
    "cramers": {"calculate": False},
    }
)
profile.to_file(os.path.join(explorations_path, 'profile_data_raw.html'))


In [ ]:
n_feature=10

def calc_elbowdata(data, base_col): 
    plotdata = data.copy().loc[~data.Income.isna(),:].groupby(base_col).size().sort_values(ascending=False)
    plotdata = pd.DataFrame(plotdata / plotdata.sum()).cumsum().rename(columns={0:'nobs_rel'})
    return plotdata

def get_feature_imp_by_expl(data, base_col, n_feature=n_feature): 
    
    # make sure every combination of levels exist, fill with 0 if no obs
    base = data[base_col].unique()
    Income = [0, 1]
    idx = pd.MultiIndex.from_product(
        [base, Income],
        names=[base_col, 'Income']
    )

    pd1 = pd.DataFrame(index=idx)
    

    a = data.groupby([base_col, 'Income']).size().to_frame().rename(columns={0:'nobs'})
    
    a = pd.concat([pd1, a], axis=1)
    a.loc[a.nobs.isna(), 'nobs'] = 0
    
    a['nobs_rel'] = a.groupby(level=base_col).transform(lambda x: x / (x[0] + x[1]))
    value_cols = a.columns.to_list()
    a.reset_index(inplace=True)
    
    # top Features by nobs: 
    topFeat = data.groupby(base_col).size().to_frame().rename(columns={0:'nobs'})\
        .sort_values('nobs', ascending=False).iloc[0:n_feature,:]\
        .index.to_list()
    
    a.sort_values(['nobs', base_col], ascending=False, inplace=True)
    
    #print('len(topFeat)', len(topFeat))
    #print('len(a)', len(a))
    #print('n_feature', n_feature)
    #print('a[base_col].nunique()', a[base_col].nunique())

    if len(topFeat) < a[base_col].nunique(): 
        print(f'***Features Filtered to top_{n_feature} by nobs!***')
    #print(a[base_col].nunique())

    a = a.loc[a[base_col].isin(topFeat),:]
    return a, value_cols

def plot_feature_imp_by_expl(data, base_col, saveplots): 
    
    a, value_cols = get_feature_imp_by_expl(data, base_col)
    
    value_cols.remove('nobs')
    print(value_cols)
    n_plots = len(value_cols) + 1

    fig, ax = plt.subplots() #, figsize=(20,7)
    #sns.barplot(data=a, x=base_col, y='nobs', hue='Income', ax=ax)#.set_title(col) # [0:10]
    #ax.tick_params(labelrotation=45)
    plotdata = calc_elbowdata(data, base_col)
    sns.lineplot(data=plotdata, y=plotdata.index, x='nobs_rel', ax=ax)
    plt.tight_layout()

    if saveplots: 
        plt.savefig(os.path.join(explorations_path, 'feature_imp_by_expl_abs.png'), dpi=200)
    
    plt.show()


    fig, ax = plt.subplots(ncols = n_plots, gridspec_kw={'width_ratios': [3,1]}) #, figsize=(20,7)
    for i, col in enumerate(value_cols): 
        sns.barplot(data=a, x=base_col, y=col, hue='Income', ax=ax[i])#.set_title(col) # [0:10]
        ax[i].tick_params(labelrotation=45)

    sns.countplot(data=data, x='Income', ax=ax[n_plots-1])
    plt.tight_layout()
    if saveplots: 
        plt.savefig(os.path.join(explorations_path, 'feature_imp_by_expl_rel.png'), dpi=200)
    plt.show()
    
def get_target_ratio(data):  
    a = data.groupby(['Income']).size().to_frame().rename(columns={0:'nobs'})
    a['nobs_rel'] = a.transform(lambda x: x / (x[0] + x[1]))
    value_cols = a.columns.to_list()
    a.reset_index(inplace=True)
    return a.loc[a.Income == 1, 'nobs_rel'].to_list()[0]


def get_feature_imp_by_target_ratio(data, base_col, weighted=False, saveplots=False): 

    target_ratio = get_target_ratio(data)
    target_ratio
                                            
    a, _ = get_feature_imp_by_expl(data, base_col, n_feature=100)
    
    #########
    nObsPerFeatClass =  data.groupby([base_col]).size().to_frame().rename(columns={0:'nobs'})
                           

    ratio_per_level = a.loc[a.Income == 1, [base_col,'nobs_rel']]\
        .set_index(base_col)\
        .rename(columns={'nobs_rel':'class1_ratio'})

    ratio_per_level = pd.concat([ratio_per_level, nObsPerFeatClass], axis=1)
    #min_max_scaler_obs = MinMaxScaler()
    ratio_per_level['nobs_rel'] = ratio_per_level.nobs / sum(ratio_per_level.nobs)


    
    ratio_per_level['diff_to_target'] = ratio_per_level['class1_ratio'] - target_ratio
    ratio_per_level['diff_to_target_dir'] = ['neg' if obs < 0 else 'pos' for obs in ratio_per_level['diff_to_target']]
    
    if weighted: 
        weights = np.power(ratio_per_level['nobs_rel'], 1./3)
    else: 
        weights = 1
        
    ratio_per_level['diff_to_target_abs'] = abs(ratio_per_level['diff_to_target']) * weights
    
    #print(ratio_per_level)

    ratio_per_level.sort_values('diff_to_target_abs', ascending=False, inplace=True)
    ratio_per_level['diff_to_target_abs_cumsum'] = ratio_per_level.diff_to_target_abs.cumsum()
    ratio_per_level


    x = ratio_per_level['diff_to_target_abs_cumsum'].values.reshape(-1, 1) #df.values #returns a numpy array
    min_max_scaler = MinMaxScaler()
    ratio_per_level['diff_to_target_abs_cumsum_scaled'] = min_max_scaler.fit_transform(x)

    print('TargetClass1_ratio', target_ratio)
    diff_to_target_df = ratio_per_level.copy()[['diff_to_target_dir', 'diff_to_target_abs']]
    diff_to_target_df.diff_to_target_abs = round(diff_to_target_df.diff_to_target_abs, 4)
    print(diff_to_target_df)
    if saveplots:
        diff_to_target_df.to_excel(os.path.join(explorations_path, 'diff_to_target_df.xlsx'))

    
    ratio_per_level.index.set_names(base_col, inplace=True)
    return ratio_per_level


def plot_feature_imp_by_target_ratio(data, base_col, weighted=False, saveplots=False): 

    r = get_feature_imp_by_target_ratio(data, base_col, weighted, saveplots=saveplots)

    sns.lineplot(data=r, y=r.index, x='diff_to_target_abs_cumsum_scaled')
    plt.tight_layout()
    
    if saveplots: 
        plt.savefig(os.path.join(explorations_path, 'feature_imp_by_target_ratio.png'), dpi=200)
    plt.show()
    
    
def plot_feature_imp_by_tree(data, base_col, n_feature=n_feature, saveplots=False): 
    # prepare
    onehot = OneHotEncoder()
    X_train_cat = data.loc[:,[base_col]]
    #X_train_cat = data[base_col]

    X_train_onehot = onehot.fit_transform(X_train_cat)
    X_train_onehot_df = pd.DataFrame(X_train_onehot.toarray(), columns=onehot.get_feature_names())
    X_train_onehot_df

    X_train_onehot_df = pd.get_dummies(data[base_col], prefix=base_col)

    # train
    dt_gini = DecisionTreeClassifier(random_state = 1)
    X_train = X_train_onehot_df#.drop(columns=['x0_Africa','x0_Europe', 'x0_Oceania'])
    y_train = data.Income


    dt_gini.fit(X_train, y_train) # data[base_col]
    print('Score:', dt_gini.score(X_train, y_train))

    #dt_gini.feature_importances_
    #tree.plot_tree(dt_gini)

    #plt.barh(onehot.get_feature_names(), dt_gini.feature_importances_)

    #print(dt_gini.feature_importances_)
    sorted_idx = dt_gini.feature_importances_.argsort()#[0:10]
    plotdata = pd.DataFrame({
        'Feature': X_train.columns[sorted_idx], 
        'Importance': dt_gini.feature_importances_[sorted_idx]}).sort_values('Importance', ascending=False)
    #plt.barh()
    #print(plotdata)
    sns.barplot(data=plotdata.iloc[0:n_feature,:], x='Importance', y='Feature')
    plt.xlabel("Feature Importance")
    plt.tight_layout()

    if saveplots: 
        plt.savefig(os.path.join(explorations_path, 'feature_imp_by_tree.png'), dpi=200)
    plt.show()

    
def plot_feature_imp(data, base_col, force_barplot=True, weighted=False, saveplots=False): 
    print('Class distributions')
    if (data[base_col].nunique() < 6) | force_barplot:
        n_plots = 2
        plot_feature_imp_by_expl(data, base_col, saveplots=saveplots)
    else: 
        n_plots = 1
        
    print('\nElbow')        
    plot_feature_imp_by_target_ratio(data, base_col, weighted, saveplots=saveplots)
    
    print('\nDecision Tree')
    plot_feature_imp_by_tree(data, base_col, saveplots=saveplots)
    
def levels_equal(data, base_col):
    train_levels = np.sort(data.loc[~data.Income.isna(), base_col].unique()).tolist()
    test_levels = np.sort(data.loc[data.Income.isna(), base_col].unique()).tolist()
    
    equal = train_levels == test_levels
    if not equal: 
        test_levels_df = pd.DataFrame({
            'test_levels': test_levels
        })

        train_levels_df = pd.DataFrame({
            'train_levels': train_levels
        })

        compare = pd.merge(left=train_levels_df, right=test_levels_df, how='outer', left_on='train_levels', right_on='test_levels')
        print(compare.loc[(compare.train_levels.isna()) | (compare.test_levels.isna()) ])

        print(f'Levels of "{base_col}" differ between test and train set')
        #raise ValueError(f'Levels of "{base_col}" differ between test and train set')
    else: 
        print('Levels ok')
    return None


In [ ]:
# 
data.info()

In [ ]:
# init 
cols_to_drop = []
cols_to_onehot = []
cols_numeric = []
cols_bool = []



# prep
pred_config = {
    'cardinality': 'original' # low, medium, high, original
} 

cardinality = pred_config['cardinality']
print('cardinality:', cardinality)

error_log = {'cleaning': []}


In [ ]:
# extract gender from name?!
base_col = 'Name'
target_col = 'male'

salutation = data[base_col].str.split(' ', n=1, expand=True)[0]
if salutation.nunique() != 3: 
    raise ValueError('Unexpected levels of salutation')
    
print(salutation.value_counts())

#gender = ['male' if s == 'Mr.' else 'female' for s in salutation]
#data['gender'] = gender

male = [1 if s == 'Mr.' else 0 if s in ['Mrs.', 'Miss'] else np.nan for s in salutation]
data[target_col] = male

if data.male.isna().sum() > 0: 
    raise Warning('NAs instroduced')


sns.countplot(data=data, hue=data.Income, x=target_col)#.set_title(col)
plt.show()
    
cols_to_drop.append(base_col)
cols_bool.append(target_col)


In [ ]:
# Compute age from Birthday

# clean whitespaces
data.Birthday = data.Birthday.str.replace(' ', '')
# define date format
dob_format = '%B%d,%Y'

# transform Birthday to datetime, catching the leap year error 

## helper fct to subtract one day from datetime if error occurs
def subone(obj):
    val = int(obj.group(0))
    return str(val-1)

## init and loop over dates
dob = []
warn_log = []
for i, d in enumerate(data.Birthday): 
    try: 
        dob.append(datetime.strptime(d, dob_format).date())

    except ValueError as e: 
        if str(e) == 'day is out of range for month': 
            dt = datetime.strptime(re.sub('\d{1,2}', subone, d, count=1), dob_format).date()
            warn_log.append((d, dt))
            dob.append(dt)
        else: 
            raise NotImplementedError('Do not know how to deal with that error!')
            dt = np.nan
            warn_log.append((d, dt))
            dob.append(dt)
        
# add age column 
data['age'] = [np.floor((datetime.strptime('2048-12-31', '%Y-%m-%d').date() - d).days / 365.2425) for d in dob]

# inspect
sns.histplot(data, x='age')
plt.show()
print('Min age:' , min(data.age))

# drop date col 
cols_to_drop.append('Birthday')
cols_numeric.append('age')


In [ ]:
data.info()

In [ ]:
data.loc[~data.Income.isna()]

In [ ]:

# 'Native Continent' to bin 
base_col = 'Native Continent'
#sns.countplot(data=data, hue=data.Income, x=base_col)#.set_title(col)
#plt.show()

plot_feature_imp(data.loc[~data.Income.isna()], base_col, weighted=False, saveplots=False)
levels_equal(data, base_col)

#low, medium, high, original
try: 
    if cardinality in ['low', 'medium']:
        target_col = 'from_europe_or_asia'
        #data['from_europe'] = [1 if a == 'Europe' else 0 for a in data[base_col]]
        data[target_col] = [1 if a in ['Europe', 'Asia'] else 0 for a in data[base_col]]
        cols_bool.append(target_col)
    elif cardinality == 'original':
        target_col = 'native_continent'
        data[target_col] = data[base_col]
        cols_to_onehot.append(target_col)
    else: 
        raise NotImplementedError(f'Can not interpret cardinality "{cardinality}" for base feature "{base_col}"!')
except Exception as e:
    error_log['cleaning'].append(e)
    raise Warning(e)
    

sns.countplot(data=data, x=target_col, hue='Income')
plt.show()

cols_to_drop.append(base_col)



In [ ]:
# Marital Status
base_col = 'Marital Status'
#target_col = 'marital_status'

data[base_col].value_counts()

plot_feature_imp(data.loc[~data.Income.isna()], base_col, weighted=False)
levels_equal(data, base_col)

try: 
    if cardinality == 'low': 
        target_col = 'maritalStatus_married'
        data[target_col] = [1 if a in ['Married', 'Married - Spouse in the Army'] else 0 for a in data[base_col]]
        cols_bool.append(target_col)
        
    elif cardinality == 'medium': 
        target_col = 'maritalStatus'
        mapping = {
            'Married':'Married',
            'Single':'Single',
            'Divorced':'Divorced',
            'Separated':'Separated',
            'Widow':'Widow',
            'Married - Spouse Missing':'SpouseMissing',
            'Married - Spouse in the Army':'Married'
        }

        data[target_col] = data[base_col].map(mapping)
        cols_to_onehot.append(target_col)
        
    elif cardinality == 'original': 
        target_col = 'maritalStatus'
        mapping = {
            'Married':'Married',
            'Single':'Single',
            'Divorced':'Divorced',
            'Separated':'Separated',
            'Widow':'Widow',
            'Married - Spouse Missing':'SpouseMissing',
            'Married - Spouse in the Army':'MarriedArmy'
        }

        data[target_col] = data[base_col].map(mapping)
        cols_to_onehot.append(target_col)

    else: 
        raise NotImplementedError(f'Can not interpret cardinality "{cardinality}" for base feature "{base_col}"!')
except Exception as e:
    error_log['cleaning'].append(e)
    raise Warning(e)
    
    
#sns.countplot(data=data, x=target_col)
sns.countplot(data=data, x=target_col, hue='Income')
plt.show()

cols_to_drop.append(base_col)


In [ ]:
# Lives with
base_col = 'Lives with'
print(data[base_col].value_counts())

plot_feature_imp(data.loc[~data.Income.isna()], base_col, weighted=False, saveplots=True)
levels_equal(data, base_col)

try: 
    if(cardinality == 'low'): 
        target_col = 'household_livesWithPartner'
        data[target_col] = [1 if a in ['Wife', 'Husband'] else 0 for a in data[base_col]]
        cols_bool.append(target_col)
    elif(cardinality == 'medium'): 
        target_col = 'household'
        mapping = {
            'Wife': 'Partner',
            'Other Family': 'Family',
            'Children': 'Children',
            'Alone': 'Alone',
            'Husband': 'Partner',
            'Other relatives': 'Family'
        }

        print(mapping)

        data[target_col] = data[base_col].map(mapping)
        cols_to_onehot.append(target_col)
    elif(cardinality == 'original'): 
        target_col = 'household'
        mapping = {
            'Wife': 'Wife',
            'Other Family': 'Family',
            'Children': 'Children',
            'Alone': 'Alone',
            'Husband': 'Husband',
            'Other relatives': 'Other'
        }

        print(mapping)

        data[target_col] = data[base_col].map(mapping)
        cols_to_onehot.append(target_col)
    else: 
        raise NotImplementedError(f'Can not interpret cardinality "{cardinality}" for base feature "{base_col}"!')
except Exception as e:
    error_log['cleaning'].append(e)
    raise Warning(e)

sns.countplot(data=data, x=target_col, hue='Income')
plt.show()

cols_to_drop.append(base_col)


In [ ]:

# 'Base Area' to bin 
base_col = 'Base Area'

plot_feature_imp(data.loc[~data.Income.isna()], base_col, weighted=False)
levels_equal(data, base_col)

try: 
    if cardinality == 'low': 
        target_col = 'basearea_fanfoss' # basearea_northbury
        target_val = 'Fanfoss'
        target_col_alt = 'basearea_northbury'
        target_val_alt = 'Northbury'

        print('\nResult:')
        data[target_col] = [1 if a == target_val else 0 for a in data[base_col]]

        print('\nAlternative result:')
        test = data[['Income', base_col]].copy()
        test[target_col_alt] = [1 if a == target_val_alt else 0 for a in test[base_col]]
        sns.countplot(data=test, x=target_col_alt, hue='Income')
        plt.show()
        cols_bool.append(target_col)
    elif cardinality == 'medium':
        target_col = 'basearea'
        data[target_col] = [
            target_val if a == target_val 
            else target_val_alt if a == target_val_alt 
            else 'Rest' for a in data[base_col]]
        cols_to_onehot.append(target_col)

    elif cardinality == 'original':
        target_col = 'basearea'
        data[target_col] = data[base_col]
        cols_to_onehot.append(target_col)
    else: 
        raise NotImplementedError(f'Can not interpret cardinality "{cardinality}" for base feature "{base_col}"!')
except Exception as e:
    error_log['cleaning'].append(e)
    raise Warning(e)

    
sns.countplot(data=data, x=target_col, hue='Income')
plt.show()

cols_to_drop.append(base_col)

In [ ]:
# Education Level 
base_col = 'Education Level'
target_col = 'education'
print(data.columns)

plot_feature_imp(data.loc[~data.Income.isna()], base_col, weighted=False)
levels_equal(data, base_col)

edu_mapping = pd.read_csv(os.path.join(src_path, 'edu_mapping.csv'), sep=';')
mapping_options = ['level_0', 'level_1', 'numeric', 'original', 'low']


#low, medium, high, original
try: 
    if cardinality == 'low':
        m_option = mapping_options[4]
        plot_fct = sns.countplot
    elif cardinality == 'medium': 
        m_option = mapping_options[4]
        plot_fct = sns.countplot
    elif cardinality == 'high': 
        m_option = mapping_options[2]
        plot_fct = sns.histplot
    elif cardinality == 'original':
        m_option = mapping_options[3]
        plot_fct = sns.countplot       
    else: 
        raise NotImplementedError(f'Can not interpret cardinality "{cardinality}" for base feature "{base_col}"!')
except Exception as e:
    error_log['cleaning'].append(e)
    raise Warning(e)
    


#print(data[base_col].value_counts())

#mapping = dict(edu_mapping[['name', mapping_options[2]]].set_index('name'))
#mapping = {k:v for k,v in edu_mapping[['name', mapping_options[2]]].set_index('name').items()}
#mapping = edu_mapping[['name', mapping_options[2]]].set_index('name')
mapping = edu_mapping[['name', m_option]].rename(columns={m_option: target_col})
print(mapping)

# drop if reruning the cell 
if target_col in data.columns: 
    data.drop(columns=[target_col], inplace=True)

data = data.merge(mapping, left_on=base_col, right_on='name', how='left')
data.drop(columns=['name'], inplace=True)  

# plot target col against prediction classes
fig, ax = plt.subplots(figsize=(15,7))
plot_fct(data=data, x=target_col, hue='Income', ax=ax)
#plt.xticks(rotation=45)
plt.show()

cols_to_drop.append(base_col)
cols_to_onehot.append(target_col)


data[[base_col, target_col]]

In [ ]:
# years of education 
base_col = 'Years of Education'
target_col = 'education_years'
#data.rename(columns={base_col: target_col}, inplace=True)
data[target_col] = data[base_col]

cols_to_drop.append(base_col)
cols_numeric.append(target_col)
data.head()
sns.histplot(data=data, x=target_col, hue='Income')
plt.show()

In [ ]:
# Employment Sector
base_col = 'Employment Sector'
target_col = 'empl_sector'

plot_feature_imp(data.loc[~data.Income.isna()], base_col, weighted=False)
#levels_equal(data, base_col)

print(data[base_col].value_counts())

#low, medium, high, original
try: 
    if cardinality == 'deprecated':
        mapping = {
            'Private Sector - Services ': 'private',
            'Self-Employed (Individual)': 'self',
            'Public Sector - Others': 'public',
            '?': 'unknown',
            'Private Sector - Others': 'private',
            'Self-Employed (Company)': 'self',
            'Public Sector - Government': 'public',
            'Unemployed': 'delete',
            'Never Worked': 'delete'
            }

    elif cardinality in ['low', 'medium', 'original']: 
        mapping = {
            'Private Sector - Services ': 'private_services',
            'Self-Employed (Individual)': 'self_individual',
            'Public Sector - Others': 'public_others',
            '?': 'unknown',
            'Private Sector - Others': 'private_others',
            'Self-Employed (Company)': 'self_company',
            'Public Sector - Government': 'public_gov',
            'Unemployed': 'unemployed',
            'Never Worked': 'unemployed'
            }
    else: 
        raise NotImplementedError(f'Can not interpret cardinality "{cardinality}" for base feature "{base_col}"!')
except Exception as e:
    error_log['cleaning'].append(e)
    raise Warning(e)


print(mapping)
    
data[target_col] = data[base_col].map(mapping)
levels_equal(data, target_col)


fig, ax = plt.subplots(figsize=(15,7))
sns.countplot(data=data, x=target_col, hue='Income', ax=ax)
plt.show()

cols_to_drop.append(base_col)
cols_to_onehot.append(target_col)

In [ ]:
# role
base_col = 'Role'
target_col = 'empl_role'

plot_feature_imp(data.loc[~data.Income.isna()], base_col, weighted=False)
levels_equal(data, base_col)

#low, medium, high, original
try: 
    if cardinality in ['low', 'medium']:
        mapping = {
            'Professor': 'Professor',
            'Management': 'Management',
            'Repair & constructions': 'Operational_low',
            'Administratives': 'Operational',
            'Sales': 'Sales',
            'Other services': 'Services',
            'Machine Operators & Inspectors': 'Operational',
            '?': 'unknown',
            'Transports': 'Operational_low',
            'Cleaners & Handlers': 'Cleaners',
            'Agriculture and Fishing': 'Operational',
            'IT': 'IT_Security',
            'Security': 'IT_Security',
            'Household Services': 'Household',
            'Army': 'Operational_low'
        }
    elif cardinality == 'original':       
        mapping = {
            'Professor': 'Professor',
            'Management': 'Management',
            'Repair & constructions': 'Constructions',
            'Administratives': 'Administratives',
            'Sales': 'Sales',
            'Other services': 'Services',
            'Machine Operators & Inspectors': 'Operator',
            '?': 'unknown',
            'Transports': 'Transports',
            'Cleaners & Handlers': 'Cleaners',
            'Agriculture and Fishing': 'Agriculture',
            'IT': 'IT', 
            'Security': 'Security',
            'Household Services': 'Household',
            'Army': 'Army'
        }
    else: 
        raise NotImplementedError(f'Can not interpret cardinality "{cardinality}" for base feature "{base_col}"!')
except Exception as e:
    error_log['cleaning'].append(e)
    raise Warning(e)

print(data[base_col].value_counts())

print(mapping)
    
data[target_col] = data[base_col].map(mapping)


fig, ax = plt.subplots(figsize=(15,7))
sns.countplot(data=data, x=target_col, hue='Income', ax=ax)
plt.show()

cols_to_drop.append(base_col)
cols_to_onehot.append(target_col)

In [ ]:
 

# Working Hours per week
base_col = 'Working Hours per week'
target_col = 'working_hrs_week'

data[target_col] = data[base_col]
#data.rename(columns={base_col: target_col}, inplace=True)

#setting working hours per week of those that have never worked or are unemployed to 0
data.loc[data['empl_sector'] == 'unemployed', 'working_hrs_week'] = 0

sns.histplot(data=data, x=target_col, hue='Income', bins=30)
plt.show()

cols_to_drop.append(base_col)
cols_numeric.append(target_col)
data.head()



In [ ]:
# Money Received
base_col = 'Money Received'
target_col = 'group_b_received_money'


#low, medium, high, original
try: 
    if cardinality == 'low':
        data[target_col] = [1 if v != 0 else 0 for v in data[base_col]]
        plot_fct = sns.countplot
        cols_bool.append(target_col)
    elif cardinality == 'original':
        data[target_col] = data[base_col]
        plot_fct = sns.histplot
        cols_numeric.append(target_col)
    else: 
        raise NotImplementedError(f'Can not interpret cardinality "{cardinality}" for base feature "{base_col}"!')
except Exception as e:
    error_log['cleaning'].append(e)
    raise Warning(e)

cols_to_drop.append(base_col)


plot_fct(data=data, x=target_col, hue='Income')
plt.show()

#data[[base_col, target_col]]



In [ ]:
   

# Ticket Price
base_col = 'Ticket Price'
target_col = 'group_c_payed'

#low, medium, high, original
try: 
    if cardinality == 'low':
        data[target_col] = [1 if v != 0 else 0 for v in data[base_col]]
        sns.countplot(data=data, x=target_col, hue='Income')
        cols_bool.append(target_col)
    elif cardinality == 'original':
        data[target_col] = data[base_col]
        sns.histplot(data=data, x=target_col, hue='Income', bins = 30)
        cols_numeric.append(target_col)
    else: 
        raise NotImplementedError(f'Can not interpret cardinality "{cardinality}" for base feature "{base_col}"!')
except Exception as e:
    error_log['cleaning'].append(e)
    raise Warning(e)
    

cols_to_drop.append(base_col)


plt.show()

#data[[base_col, target_col]]

In [ ]:
# Check for errors
for name, log in error_log.items():
    if len(log) > 0: 
        print(f'{name}:\n {log}')
        raise Warning('Errors occured! See above.')

In [ ]:
error_log

In [ ]:
# drop cols
#cols_to_drop.append('CITIZEN_ID')
col_with_index = ['CITIZEN_ID']
data.drop(columns=cols_to_drop, inplace=True)


In [ ]:
combined_data = data.copy().set_index(col_with_index)


In [ ]:
## profile report

create_cleaning_report = False
if create_cleaning_report: 
    profile = ProfileReport(
        data,
        title=f'Cleaned data {dataset_name}' ,
        minimal=False, 
        correlations={
        "pearson": {"calculate": True},
        "spearman": {"calculate": False},
        "kendall": {"calculate": False},
        "phi_k": {"calculate": False},
        "cramers": {"calculate": False},
        }
    )
    profile.to_file(os.path.join(explorations_path, f'profile_data_cleaned_{dataset_name}.html'))


In [ ]:
combined_data.shape

In [ ]:
test_data = combined_data.copy().loc[combined_data.Income.isna()]
test_data.drop(columns='Income', inplace=True)
test_data.info()

data = combined_data.copy().loc[~combined_data.Income.isna()]
data.Income = data.Income.astype(int)
data.info()




# Explorations

In [ ]:
plot_explorations = True

In [ ]:
data.isna().sum()

In [ ]:
data.info()

In [ ]:
print(data.shape[1])
data.shape[1] - (len(cols_numeric) + len(cols_bool) + len(cols_to_onehot))

In [ ]:
# check colmuns
expected_ncols = len(cols_numeric) + len(cols_bool)
for cat_col in cols_to_onehot: 
    expected_ncols += data[cat_col].nunique()
    
expected_ncols

In [ ]:
print(len(cols_to_drop))
cols_to_drop

In [ ]:
# target distribution

sns.countplot(data=data, x='Income')
plt.show()

In [ ]:
if plot_explorations: 
    # Prepare figure
    my_dpi = 200
    fig = plt.figure(
        figsize=(
            #10, 8
            1000/my_dpi, 1000/my_dpi
        )
    ) 

    # Obtain correlation matrix. Round the values to 2 decimal cases. Use the DataFrame corr() and round() method.
    corr = np.round(data.corr(method="pearson"), decimals=2)

    # Build annotation matrix (values above |0.5| will appear annotated in the plot)
    mask_annot = np.absolute(corr.values) >= 0.5
    annot = np.where(mask_annot, corr.values, np.full(corr.shape,"")) # Try to understand what this np.where() does

    # Plot heatmap of the correlation matrix
    sns.heatmap(data=corr, annot=annot, cmap=sns.diverging_palette(220, 10, as_cmap=True), 
                fmt='s', vmin=-1, vmax=1, center=0, square=True, linewidths=.5)

    # Layout
    fig.subplots_adjust(top=0.95)
    # fig.suptitle("Correlation Matrix", fontsize=20)

    plt.tight_layout()

    plt.savefig(
        os.path.join(explorations_path, 'correlation_matrix.png')
        ,dpi=my_dpi
    )
    plt.show()

In [ ]:
# distributions 
if plot_explorations: 
    plotdata = data.loc[:,cols_numeric + cols_bool]
    ncols = 3
    n_plots = plotdata.shape[1]
    nrows = int(np.ceil(n_plots/ncols))


    my_dpi = 200

    fig, ax = plt.subplots(
        ncols=ncols, nrows=nrows, 
        figsize=(
            # 15,13
            1200/my_dpi, 1000/my_dpi
        )
    )
    col_no = 0
    for i in range(nrows):
        for j in range(ncols): 
            if col_no < n_plots:
                col = plotdata.columns[col_no]
                print(col)
                if data[col].dtype in [np.float, np.int]: 
                    sns.histplot(data=plotdata, hue=data.Income, x=col, ax=ax[i,j], bins=30).set_title(col)
                else : 
                    sns.countplot(data=plotdata, hue=data.Income, x=col, ax=ax[i,j], dodge=True).set_title(col)
                ax[i,j].tick_params(labelrotation=45)
                col_no +=1

    fig.tight_layout()
    
    plt.savefig(os.path.join(explorations_path, 'distributions.png'), dpi=200)
    plt.show()


# Data Preparation

In [ ]:
# prep config

prep_config = {
    'overSampling': False, 
    'scale': 'minmax', # standard, minmax,
    'feature_selection':False#'boruta'
}

In [ ]:
# split the cleaned data into test and train data 
X_train = data.copy().drop(['Income'], axis=1)#.values
y = data.copy().loc[:,'Income'].values

X_test = test_data.copy()#.values
X_test.shape

print('X_train.shape', X_train.shape)
print('y.shape', y.shape)
print('X_test.shape', X_test.shape)

In [ ]:
X_train.info()
X_test.info()

## One hot encode and scale

In [ ]:
# https://towardsdatascience.com/feature-selection-with-borutapy-f0ea84c9366

def onehot_scale(X_train, X_test, scaler, verbose=False): 
    ###Creating series for categorical test and train
    X_train_bool = X_train[cols_bool]
    X_test_bool = X_test[cols_bool]

    ###Instantiating One Hot Encoder
    ohe = OneHotEncoder()
    ###Creating series for categorical test and train
    X_train_cat = X_train[cols_to_onehot]
    X_test_cat = X_test[cols_to_onehot]
    ###Fitting encoder to training categorical features and transforming ###test and train
    X_train_ohe = ohe.fit_transform(X_train_cat)
    X_test_ohe = ohe.transform(X_test_cat)

    ###Converting series to dataframes
    columns = ohe.get_feature_names(input_features=X_train_cat.columns)
    X_train_processed = pd.DataFrame(X_train_ohe.todense(), columns=columns, index=X_train_bool.index)
    X_test_processed = pd.DataFrame(X_test_ohe.todense(), columns=columns, index=X_test_bool.index)


    ###Instantiating Standard Scaler
    if scaler == 'standard':        
        ss = StandardScaler()
    elif scaler == 'minmax': 
        ss = MinMaxScaler()
    else: 
        raise ValueError(f'Can not interpret {scaler} as scaler!')

    ###Converting continuous feature values to floats
    X_train_cont = X_train[cols_numeric].astype(float)
    X_test_cont = X_test[cols_numeric].astype(float)
    ###Fitting scaler to training continuous features and transforming ###train and test
    X_train_scaled = ss.fit_transform(X_train_cont)
    X_test_scaled = ss.transform(X_test_cont)


    ###Concatenating scaled and encoded dataframes
    X = pd.concat([pd.DataFrame(X_train_scaled, index=X_train_bool.index), X_train_bool, X_train_processed], axis=1).values
    X_test_ = pd.concat([pd.DataFrame(X_test_scaled, index=X_test_bool.index),X_test_bool, X_test_processed], axis=1).values
    
    if verbose: 
        print('X_train_scaled.shape', X_train_scaled.shape)
        print('X_train_bool.shape', X_train_bool.shape)
        print('X_train_processed.shape', X_train_processed.shape)

        print('X.shape', X.shape)
        print('y.shape', y.shape)
        print('X_test.shape', X_test_.shape)
    
    return X, X_test_

if True: 
    X, X_test_ = onehot_scale(X_train, X_test, scaler=prep_config['scale'])


## feature_selection

In [ ]:
# https://github.com/scikit-learn-contrib/boruta_py
# https://towardsdatascience.com/feature-selection-with-borutapy-f0ea84c9366


# load X and y
# NOTE BorutaPy accepts numpy arrays only, hence the .values attribute
# X = pd.read_csv('examples/test_X.csv', index_col=0).values
# y = pd.read_csv('examples/test_y.csv', header=None, index_col=0).values
# y = y.ravel()

if prep_config['feature_selection'] == 'boruta': 


    # define random forest classifier, with utilising all cores and
    # sampling in proportion to y labels
    rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)

    # define Boruta feature selection method
    feat_selector = BorutaPy(rf, n_estimators='auto', verbose=0, random_state=1)

    # find all relevant features - 5 features should be selected
    feat_selector.fit(X, y)

    # check selected features - first 5 features are selected
    print(feat_selector.support_)
    # check ranking of features
    print(feat_selector.ranking_)

    # call transform() on X to filter it down to selected features
    X_filtered = feat_selector.transform(X)
    X_test_filtered = feat_selector.transform(X_test_)

    
    X_metadata = pd.DataFrame({
    'Features': (X_train.drop('Income', axis=1).columns.to_list()),
    'support': (feat_selector.support_), 
    'ranking': (feat_selector.ranking_)
    })

    X_metadata
    
    X_metadata.support.sum()

## overSampling to cope with class imbalance 

In [ ]:
## Train test split on updated X
X_t, X_val, y_t, y_val = train_test_split(X, y, random_state=42, stratify=None)
sns.countplot(y_t)
plt.show()

# https://imbalanced-learn.org/stable/generated/imblearn.over_sampling.SMOTE.html
if prep_config['overSampling']: 
    sm = SMOTE(random_state=2, n_jobs=-1, k_neighbors=5, sampling_strategy='auto')
    X_t, y_t = sm.fit_sample(X_t, y_t)
    sns.countplot(y_t)
    plt.show()

# Modelling

In [ ]:
# define wich models to run
modelling_config = {
    'naive_bayes1': False,
    'naive_bayes2': False,
    'naive_bayes3': True,
    'logistic_regression': False,
    'knn': False,
    'bagging_knn': False,
    'svc': False,
    'dt': False,
    'random_forest': False, 
    'boosted_tree': False, 
    'boosted_tree_gs': False,
    'boosted_tree_after_gs': False,
    'mlp': False,
    'mlp_gs': False,
    'mlp_after_gs': False, 
    'ensemble1': False,
    'ensemble2': False,
    'adaboost': False,
    'stacking1': False,
    'stacking2' : False,
    'stacking3' : False
}

model_comparison = [
    'naive_bayes1',
    'naive_bayes2',
    'naive_bayes3',
    'logistic_regression',
    'knn',
    'bagging_knn',
    'svc',
    'dt',
    'random_forest', 
    'boosted_tree', 
    'boosted_tree_gs',
    'boosted_tree_after_gs',
    'mlp',
    'mlp_gs',
    'mlp_after_gs', 
    'ensemble1',
    'ensemble2',
    'adaboost',
    'stacking1',
    'stacking2',
    'stacking3'
]

model_comparison = [
    'naive_bayes1',
    'naive_bayes2', 
    'naive_bayes3', 
    'ensemble1'
]


In [ ]:
def save_submission_csv(model, X_test_, test_data, file_no): 
    test_pred = model.predict(X_test_)
    submission_df = pd.DataFrame({'Income':test_pred}, index=test_data.index).reset_index()
    filename = f'Group26_Version{file_no}.csv'
    submission_df.to_csv(os.path.join(submissions_path, filename), index=False)

def train_model(model, params, X_t, y_t, X_val, y_val):
    clf = model(random_state=0, verbose=False, **params)

    clf.fit(X_t, y_t)
    predicted = clf.predict(X_val)

    print(classification_report(y_val, predicted))
    print('f1_micro:', f1_score(y_val, predicted, average='micro')) 
    print('Acc:', clf.score(X_val, y_val))
    return clf


In [ ]:
def train_model_simple(model, params, X_t, y_t, X_val, y_val):
    clf = model(**params)

    clf.fit(X_t, y_t)
    predicted = clf.predict(X_val)

    print(classification_report(y_val, predicted))
    print('f1_micro:', f1_score(y_val, predicted, average='micro')) 
    print('Acc:', clf.score(X_val, y_val))
    return clf

# <font color='#E8800A'>Naive Bayes</font> <a class="anchor" id="first-bullet"></a>
  [Back to TOC](#toc)

In [ ]:
###Instantiating gaussian naive bayes
naive_bayes1 = GaussianNB()

if modelling_config['naive_bayes1']: 
    ###Fitting naive bayes to train
    naive_bayes1.fit(X_t,y_t)
    ###Predicting on test data
    predicted = naive_bayes1.predict(X_val)
    ###Training Score
    naive_bayes1.score(X_t, y_t)
    ###Test Score
    naive_bayes1.score(X_val,y_val)
    
    print(classification_report(y_val, predicted))
    print('f1_micro:', f1_score(y_val, predicted, average='micro')) 
    # f1_micro: 0.48178571428571426
    # f1_micro: 0.44875

In [ ]:

###Instantiating multinomial naive bayes
naive_bayes2 = MultinomialNB()

###used data scaled with minmax here 
if modelling_config['naive_bayes2'] and prep_config['scale'] == 'minmax': 

    ###Fitting naive bayes to train
    naive_bayes2.fit(X_t,y_t)
    ###Predicting on test data
    predicted = naive_bayes2.predict(X_val)
    ###Training Score
    naive_bayes2.score(X_t, y_t)
    ###Test Score
    naive_bayes2.score(X_val,y_val)
    
    print(classification_report(y_val, predicted))
    print('f1_micro:', f1_score(y_val, predicted, average='micro')) 
    # f1_micro: 0.8044642857142859


###Instantiating complement naive bayes
naive_bayes3 = ComplementNB()

if modelling_config['naive_bayes3']: 

    ###Fitting naive bayes to train
    naive_bayes3.fit(X_t,y_t)
    ###Predicting on test data
    predicted = naive_bayes3.predict(X_val)
    ###Training Score
    naive_bayes3.score(X_t, y_t)
    ###Test Score
    naive_bayes3.score(X_val,y_val)
    
    print(classification_report(y_val, predicted))
    print('f1_micro:', f1_score(y_val, predicted, average='micro')) 
    # f1_micro: 0.7673214285714286
    
    scores = cross_val_score(naive_bayes3, X, y, cv=10, scoring='f1_micro')
    print(f'{scores.mean()} ~ {scores.std()}')
    #0.7610267857142856 ~ 0.005905863622498532



# <font color='#E8800A'>Logistic Regression</font> <a class="anchor" id="second-bullet"></a>
  [Back to TOC](#toc)

In [ ]:
if modelling_config['logistic_regression']: 
    ###Instantiating logistic regression
    log_model = LogisticRegression()
    ###Fitting logistic regression to train
    log_model.fit(X_t,y_t)
    ###Predicting on test data
    predicted = log_model.predict(X_val)
    ###Training Score
    log_model.score(X_t,y_t)
    ###Test Score
    log_model.score(X_val,y_val)
    
    print(classification_report(y_val, predicted))
    print('f1_micro:', f1_score(y_val, predicted, average='micro')) 
    # f1_micro: 0.8528571428571429

In [ ]:
if modelling_config['logistic_regression']: 
    model = LogisticRegression(random_state=0, verbose=False)
    f1w = make_scorer(f1_score, average='micro')


    parameter_space = {
        'penalty': ['l1', 'l2', 'elasticnet', 'none'],
        'C': list(range(1,10)),
        'class_weight':[{0:1.0, 1:1.0},{0:1.0, 1:50.0}],
        'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
        'multi_class':['auto','ovr','multinomial']
        #dual
        #tolerance for stopping criteria
        #max_iter
        #refit
        #intercept_scaling
        #l1_ratios
    }

    lr_gs = GridSearchCV(model, parameter_space, n_jobs=-1, scoring=f1w, cv=5, verbose=10)

    lr_gs.fit(X_t, y_t)

    print(f'{lr_gs.scoring}: {lr_gs.best_score_}')
    # make_scorer(f1_score, average=micro): 0.85


In [ ]:
# lr_gs.best_params_
#{'C': 1,
# 'class_weight': {0: 1.0, 1: 1.0},
# 'multi_class': 'auto',
# 'penalty': 'l2',
# 'solver': 'liblinear'}
lr = LogisticRegression(C=1,class_weight={0: 1.0, 1: 1.0},penalty = 'l2',solver='liblinear',multi_class='auto')

if modelling_config['logistic_regression']: 
    lr_params_short = {
        'C': 1,
        'class_weight': {0: 1.0, 1: 1.0},
        'penalty': 'l2',
        'solver': 'liblinear',
        'multi_class': 'auto'
    }

    # train final model with best_params from grid search 
    lr = train_model(
        LogisticRegression,
        lr_params_short,
    #   lr_gs.best_params_,
        X_t, y_t, X_val, y_val
    )
    # f1_micro: 0.8530357142857142

    scores = cross_val_score(
        lr, X_val, y_val, cv=10, scoring='f1_micro'
    )
    print(f'{scores.mean()} ~ {scores.std()}')
    # 0.8510714285714286 ~ 0.008151937293223808

In [ ]:
###Randomized Search

if modelling_config['logistic_regression']: 
    rsearch = RandomizedSearchCV(estimator=LogisticRegression(random_state=0, verbose=False), param_distributions=parameter_space, n_iter=100)
    rsearch.fit(X_t, y_t)
    # rsearch.best_score_): 0.8498809523809523
    lr_params_short = {
        'C': 6,
        'class_weight': {0: 1.0, 1: 1.0},
        'penalty': 'l2',
        'solver': 'saga',
        'multi_class': 'ovr'
    }

    # train final model with best_params from grid search 
    lr = train_model(
        LogisticRegression,
        lr_params_short,
    #   rsearch.best_params_ : {'solver': 'saga','penalty': 'l2','multi_class': 'ovr','class_weight': {0: 1.0, 1: 1.0},'C': 6}
        X_t, y_t, X_val, y_val
    )
    # f1_micro: 0.8528571428571429

    scores = cross_val_score(
        lr, X_val, y_val, cv=10, scoring='f1_micro'
    )
    print(f'{scores.mean()} ~ {scores.std()}')
    # 0.8514285714285714 ~ 0.008329931278350444

# <font color='#E8800A'>KNN</font> <a class="anchor" id="third-bullet"></a>
  [Back to TOC](#toc)

In [ ]:
if modelling_config['knn']: 
    ###Instantiating k nearest neighbors 
    KNNmodel = KNeighborsClassifier()
    ###Fitting k nearnest neighbors to train
    KNNmodel.fit(X_t,y_t)
    ###Predicting on test data
    predicted = KNNmodel.predict(X_val)
    ###Training Score
    KNNmodel.score(X_t,y_t)
    ###Test Score
    KNNmodel.score(X_val,y_val)
    
    print(classification_report(y_val, predicted))
    print('f1_micro:', f1_score(y_val, predicted, average='micro')) 
    # f1_micro: 0.8351785714285714

In [ ]:
if modelling_config['knn']: 
    model = KNeighborsClassifier()
    f1w = make_scorer(f1_score, average='micro')


    parameter_space = {
        'n_neighbors': list(range(1,15)),
        'weights': ['uniform','distance'],
        'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute'],
        'leaf_size' : [5,10,15,30,45] 
    }

    knn_gs = GridSearchCV(model, parameter_space, n_jobs=-1, scoring=f1w, cv=5, verbose=10)

    knn_gs.fit(X_t, y_t)

    print(f'{knn_gs.scoring}: {knn_gs.best_score_}')
    # make_scorer(f1_score, average=micro): 0.8419047619047619


In [ ]:
# knn_gs.best_params_
#{'algorithm': 'brute',
# 'leaf_size': 15,
# 'n_neighbors': 11,
# 'weights': 'uniform'}
knn = KNeighborsClassifier(algorithm = 'brute', n_neighbors = 11, weights = 'uniform')

if modelling_config['knn']: 
    knn_params_short = {
        'algorithm': 'brute',
        'n_neighbors': 11,
        'weights': 'uniform'
    }

    # train final model with best_params from grid search 
    knn = train_model_simple(
        KNeighborsClassifier,
        knn_params_short,
    #   knn_gs.best_params_,
        X_t, y_t, X_val, y_val
    )
    # f1_micro: 0.8483928571428572

    scores = cross_val_score(
        knn, X_val, y_val, cv=10, scoring='f1_micro'
    )
    print(f'{scores.mean()} ~ {scores.std()}')
    # 0.8396428571428572 ~ 0.008096988606253283

In [ ]:
# bagging knn
if modelling_config['bagging_knn']: 
    ###Instantiating k nearest neighbors 
    bagging_knn = BaggingClassifier(base_estimator = knn, random_state = 0)
    ###Fitting k nearnest neighbors to train
    bagging_knn.fit(X_t,y_t)
    ###Predicting on test data
    predicted = bagging_knn.predict(X_val)
    ###Training Score
    bagging_knn.score(X_t,y_t)
    ###Test Score
    bagging_knn.score(X_val,y_val)
    
    print(classification_report(y_val, predicted))
    print('f1_micro:', f1_score(y_val, predicted, average='micro')) 
    # f1_micro: 0.8469642857142857

# <font color='#E8800A'>Support Vector Machine</font> <a class="anchor" id="fourth-bullet"></a>
  [Back to TOC](#toc)

In [ ]:
if modelling_config['svc']: 
    ###Instantiating support vector classifier
    svc_model = SVC(gamma='scale')
    ###Fitting logistic regression to train
    svc_model.fit(X_t,y_t)
    ###Predicting on test data
    predicted = svc_model.predict(X_val)
    ###Training Score
    svc_model.score(X_t,y_t)
    ###Test Score
    svc_model.score(X_val,y_val)
    
    print(classification_report(y_val, predicted))
    print('f1_micro:', f1_score(y_val, predicted, average='micro')) 
    # f1_micro: 0.8625

In [ ]:
if modelling_config['svc']: 
    model = SVC()
    f1w = make_scorer(f1_score, average='micro')


    parameter_space = {
        'C': [0.1,0.5,1,10,100],
        'kernel': ['linear','poly','rbf','sigmoid'],
        'gamma': ['scale', 'auto']
    }

    svc_gs = GridSearchCV(model, parameter_space, n_jobs=-1, scoring=f1w, cv=5, verbose=10)

    svc_gs.fit(X_t, y_t)

    print(f'{svc_gs.scoring}: {svc_gs.best_score_}')
    # make_scorer(f1_score, average=micro): 0.85375

In [ ]:
# svc_gs.best_params_
#{'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}
svc = SVC(C = 1, gamma = 'scale', kernel = 'rbf', random_state=0, verbose=False, probability = True)


if modelling_config['svc']: 
    svc_params_short = {
        'C': 1,
        'gamma': 'scale',
        'kernel': 'rbf'
    }

    # train final model with best_params from grid search 
    svc = train_model(
        SVC,
        svc_params_short,
    #   svc_gs.best_params_,
        X_t, y_t, X_val, y_val
    )
    # f1_micro:0.8625

    scores = cross_val_score(
        svc, X_val, y_val, cv=10, scoring='f1_micro'
    )
    print(f'{scores.mean()} ~ {scores.std()}')
    # 0.8551785714285713 ~ 0.012511474325430974

# <font color='#E8800A'>Decision Trees</font> <a class="anchor" id="fifth-bullet"></a>
  [Back to TOC](#toc)

In [ ]:
if modelling_config['dt']: 
    ###Instantiating Decision Tree Classifier
    dt = DecisionTreeClassifier()
    ###Fitting Decision Tree Classifier to train and test
    dt.fit(X_t, y_t)
    ###Predicting on test data
    predicted = dt.predict(X_val)
    ###Test Score
    dt.score(X_val, y_val)
    ###Training Score
    dt.score(X_t, y_t)

    print(classification_report(y_val, predicted))
    print('f1_micro:', f1_score(y_val, predicted, average='micro')) 
    # f1_micro: 0.8139285714285716

# <font color='#E8800A'>Random Forest</font> <a class="anchor" id="sixth-bullet"></a>
  [Back to TOC](#toc)

In [ ]:
rf2 = RandomForestClassifier(n_jobs=-1, n_estimators=500, oob_score=True, max_depth=6, random_state=42)

if modelling_config['random_forest']: 
    ###Instantiating Random Forest Classifier
    ###Fitting Random Forest Classifier to train and test
    rf2.fit(X_t, y_t)
    ###Predicting on test data
    predicted = rf2.predict(X_val)
    ###Test Score
    rf2.score(X_val, y_val)
    ###Training Score
    rf2.score(X_t, y_t)

    print(classification_report(y_val, predicted))
    print('f1_micro:', f1_score(y_val, predicted, average='micro')) 
    # f1_micro: 0.8482142857142857


# <font color='#E8800A'>Boosted Trees</font> <a class="anchor" id="seventh-bullet"></a>
  [Back to TOC](#toc)

In [ ]:
clf1 = GradientBoostingClassifier(
    learning_rate=.1,
    random_state=0, verbose=False,
    subsample=1,
    n_estimators=1000, max_depth=7, min_impurity_decrease = 0.1,
    max_features='auto',
    n_iter_no_change=30, validation_fraction=0.1,
    warm_start=False
)

if modelling_config['boosted_tree']: 
    clf1.fit(X_t, y_t)
    predicted = clf1.predict(X_val)

    print(classification_report(y_val, predicted))
    print('f1_micro:', f1_score(y_val, predicted, average='micro')) 
    # f1_micro: 0.8755357142857143
    print('Acc:', clf1.score(X_val, y_val))

    save_submission_csv(clf1, X_test_, test_data, '12')
    
    scores = cross_val_score(
    clf1, X, y, cv=10, scoring='f1_micro'
    )
    print(f'{scores.mean()} ~ {scores.std()}')
    # 0.8708482142857144 ~ 0.006533890504710872

In [ ]:
if modelling_config['boosted_tree_gs']: 
    model = GradientBoostingClassifier(random_state=0, verbose=False)
    f1w = make_scorer(f1_score, average='micro')


    parameter_space = {
        'learning_rate':[.1, .05],
        'subsample':[1, .9],
        'n_estimators':[1000], 
        'min_samples_split': [2,3],
        'max_depth': list(range(5,9)), 
        'min_impurity_decrease': [0.0, 0.1],
        'max_features': ['auto', None],
        'n_iter_no_change': [30],
        'validation_fraction': [0.1],
        'warm_start': [False], 
        'ccp_alpha': [0.0]
    }


    clf_gs = GridSearchCV(model, parameter_space, n_jobs=-1, scoring=f1w, cv=5, verbose=10)

    clf_gs.fit(X, y)

    print(f'{clf_gs.scoring}: {clf_gs.best_score_}')
    # make_scorer(f1_score, average=micro): 0.8710267857142856


In [ ]:
# clf3.get_params()

if modelling_config['boosted_tree_after_gs']: 
    clf3_params_short = {
        'ccp_alpha': 0.0,
        'learning_rate': 0.1,
        'max_depth': 6,
        'max_features': 'auto',
        'min_impurity_decrease': 0.1,
        'min_samples_split': 3,
        'n_estimators': 1000,
        'n_iter_no_change': 30,
        'subsample': 1,
        'validation_fraction': 0.1,
        'warm_start': False
    }

    # train final model with best_params from grid search 
    clf3 = train_model(
        GradientBoostingClassifier,
        clf3_params_short,
    #   clf_gs.best_params_,
        X_t, y_t, X_val, y_val
    )
    # f1_micro: 0.8753571428571428

    scores = cross_val_score(
        clf3, X, y, cv=10, scoring='f1_micro'
    )
    print(f'{scores.mean()} ~ {scores.std()}')
    # 0.8708482142857144 ~ 0.007379047013092432

    # save
    save_submission_csv(clf3, X_test_, test_data, '11')


# <font color='#E8800A'>Neural Networks</font> <a class="anchor" id="eighth-bullet"></a>
  [Back to TOC](#toc)

In [ ]:
if modelling_config['mlp']: 
        model = MLPClassifier(
            alpha=0.000001,
            activation='relu', # [tanh, relu]
            random_state=0, verbose=False, 
            hidden_layer_sizes = (100, ),# [(100,), (50,50), (200,)]
            max_iter=1000, early_stopping=True, 
            learning_rate_init = 0.001,  # [0.05, 0.001, 0.005]
            learning_rate='constant', # ['invscaling', 'constant']
            momentum=0.9, #[0.8, 0.9]
            solver='adam', # ['adma', 'sgd']
            beta_1 = .9, #[.7, .8, .9]
            beta_2 = .999 #[.9, .99, .8]
        )
        model.fit(X_t, y_t)
        predicted = model.predict(X_val)

        print(classification_report(y_val, predicted))
        print('f1_micro:', f1_score(y_val, predicted, average='micro')) 
        # f1_micro: 0.8614285714285714
        print('Acc:', model.score(X_val, y_val))
        
        if True: 
                scores = cross_val_score(
                model, X, y, cv=5, scoring='f1_micro'
                )
                print(f'{scores.mean()} ~ {scores.std()}')
                # 0.8549107142857144 ~ 0.004171181652427915
            

In [ ]:
if modelling_config['mlp_gs']: 
    model = MLPClassifier(random_state=0, verbose=False, max_iter=1000, early_stopping=True)
    f1w = make_scorer(f1_score, average='micro')
   
    params = {
        'alpha': (10.0 ** -np.arange(1, 7)).tolist(), 
        'activation': ['tanh', 'relu'],  
        'hidden_layer_sizes': [(100,), (50,50), (200,), (50,50, 50 )], 
        'learning_rate_init': [0.05, 0.001, 0.005],
        'learning_rate':['adaptive', 'constant'], 
        'momentum':[0.8, 0.9], 
        'solver': ['adam'], #['adma', 'sgd']
        'beta_1': [.7, .8, .9], 
        'beta_2':[.9, .99, .8]
    }


    mlp_gs = GridSearchCV(model, params, n_jobs=-1, scoring=f1w, cv=5, verbose=10)

    mlp_gs.fit(X, y)

    print(f'{mlp_gs.scoring}: {mlp_gs.best_score_}')
    #make_scorer(f1_score, average=micro): 0.8584375


In [ ]:
mlp_gs_best_params = {
    'activation': 'tanh',
    'alpha': 0.0001,
    'beta_1': 0.9,
    'beta_2': 0.8,
    'hidden_layer_sizes': (50, 50, 50),
    'learning_rate': 'adaptive',
    'learning_rate_init': 0.001,
    'momentum': 0.8,
    'solver': 'adam'
}

mlp_after_gs = MLPClassifier(random_state=0, verbose=False, max_iter=1000, early_stopping=True, **mlp_gs_best_params)


if modelling_config['mlp_after_gs']: 
        mlp_after_gs.fit(X_t, y_t)
        predicted = mlp_after_gs.predict(X_val)

        print(classification_report(y_val, predicted))
        print('f1_micro:', f1_score(y_val, predicted, average='micro')) 
        # f1_micro: 0.8546428571428571
        print('Acc:', mlp_after_gs.score(X_val, y_val))
        

# <font color='#E8800A'>Ensembles</font> <a class="anchor" id="ninth-bullet"></a>
  [Back to TOC](#toc)

### VotingClassifier

In [ ]:
ensemble1 = VotingClassifier(
    estimators=[('rf', rf2), ('gbt', clf1), ('mlp', mlp_after_gs)],
    voting='soft'
)

if modelling_config['ensemble1']: 
    ensemble1.fit(X_t, y_t)
    predicted = ensemble1.predict(X_val)

    print(classification_report(y_val, predicted))

    ensemble1.score(X_val, y_val)
    
    if True:
        scores = cross_val_score(
            ensemble1, X, y, cv=5, scoring='f1_micro'
        )
        print(f'{scores.mean()} ~ {scores.std()}')
        #0.8653125000000002 ~ 0.0036202026665559423

In [ ]:
if modelling_config['ensemble2']: 
    eclf = VotingClassifier(
        estimators=[('gbt', clf1), ('svc', svc)],
        voting='soft')

    eclf.fit(X_t, y_t)
    predicted = eclf.predict(X_val)

    print(classification_report(y_val, predicted))

    eclf.score(X_val, y_val)
    
    if True:
        scores = cross_val_score(
            eclf, X, y, cv=5, scoring='f1_micro'
        )
        print(f'{scores.mean()} ~ {scores.std()}')
        #0.8668750000000001 ~ 0.004359484148476345

### Adaboost

In [ ]:
if modelling_config['adaboost']: 
    ab = AdaBoostClassifier(random_state = 5)

    ab.fit(X_t, y_t)
    predicted = ab.predict(X_val)

    print(classification_report(y_val, predicted))

    ab.score(X_val, y_val)
    
    if True:
        scores = cross_val_score(
            ab, X, y, cv=5, scoring='f1_micro'
        )
        print(f'{scores.mean()} ~ {scores.std()}')
        # 0.8580357142857145 ~ 0.004430677062785573

### Stacking

In [ ]:

if modelling_config['stacking1']: 
    estimators = [('gbt', clf1), ('mlp', mlp_after_gs)]
    sc = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())

    sc.fit(X_t, y_t)
    predicted = sc.predict(X_val)

    print(classification_report(y_val, predicted))

    sc.score(X_val, y_val)
    
    if True:
        scores = cross_val_score(
            sc, X, y, cv=5, scoring='f1_micro'
        )
        print(f'{scores.mean()} ~ {scores.std()}')
        #0.8653125000000002 ~ 0.0036202026665559423
        
    save_submission_csv(sc, X_test_, test_data, '13')

In [ ]:

if modelling_config['stacking2']: 
    sclf = StackingClassifier(
        estimators=[('gbt', clf1), ('mlp', mlp_after_gs)],
        final_estimator=svc)

    sclf.fit(X_t, y_t)
    predicted = sclf.predict(X_val)

    print(classification_report(y_val, predicted))

    sclf.score(X_val, y_val)
    
    if True:
        scores = cross_val_score(
            sclf, X, y, cv=5, scoring='f1_micro'
        )
        print(f'{scores.mean()} ~ {scores.std()}')
        # 0.8708482142857144 ~ 0.003944266537698527

In [ ]:

if modelling_config['stacking3']: 
    sclf = StackingClassifier(
        estimators=[('rf', rf2), ('gbt', clf1), ('mlp', mlp_after_gs), ('knn', knn)],
        final_estimator=svc)

    sclf.fit(X_t, y_t)
    predicted = sclf.predict(X_val)

    print(classification_report(y_val, predicted))

    sclf.score(X_val, y_val)
    
    if True:
        scores = cross_val_score(
            sclf, X, y, cv=5, scoring='f1_micro'
        )
        print(f'{scores.mean()} ~ {scores.std()}')
        # 0.8710714285714285 ~ 0.004056363104143051

In [ ]:
if True: 
    for model in model_comparison:
        model_inst = globals()[model]

        print(model_inst)
        scores = cross_val_score(
            model_inst, X, y, cv=10, scoring='f1_micro'
        )
        print(f'{scores.mean()} ~ {scores.std()}')